In [11]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

# Scrape daily case data from Worldometers

In [12]:
import urllib.request
from urllib.request import urlopen
from bs4 import BeautifulSoup
import requests as req

user_agent = 'Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US; rv:1.9.0.7) Gecko/2009021910 Firefox/3.0.7'
headers={'User-Agent':user_agent,}

def scraper(url, selector):
    request=urllib.request.Request(url,None,headers) #The assembled request
    html = urlopen(request)
    soup = BeautifulSoup(html, "html.parser")
    script_divs = soup.find_all('script', {'type': 'text/javascript'})
    res = 0
    for i in range(len(script_divs)):
#         print(i)
#         print(script_divs[i])
        if selector in str(script_divs[i]):
            res = script_divs[i]
            break
#     print(res)
    lst = str(res).split(',')
    lst = [item.strip() for item in lst]
    lst = [test.replace('\n', '').replace('"', '').replace('            ', '') for test in lst]
    return lst

In [13]:
def parseVals(url, country, selector):
    arr = scraper(url,selector)
    start = False
    end = False
    data_start = False
    data_end = False
    for i in range(len(arr)):
        if "xAxis" in arr[i]:
            start = i
        elif "]" in arr[i] and not data_start:
            end = i
        elif "data" in arr[i] and end:
            data_start = i
        elif "]" in arr[i] and data_start:
            data_end = i
            break
        if data_end:
            print("here")
            break
    arr[start] = arr[start].replace("xAxis: {categories: [", '')
    arr[data_start] = arr[data_start].replace("data: [", '')
    arr[end] = arr[end].replace("]        }", '')
    arr[data_end] = arr[data_end].replace("]        }]", '')
    df = pd.DataFrame()
    df["date"] = arr[start:end+1]
    df["value"] = arr[data_start:data_end+1]
    df["country"] = country
    return df

In [14]:
countries = """us
brazil
india
mexico
uk
italy
france
spain
peru
iran
colombia
argentina
russia
south-africa
chile
indonesia
ecuador
belgium
iraq
germany
canada
turkey
bolivia
netherlands
philippines
pakistan
ukraine
romania
egypt
sweden
bangladesh
saudi-arabia
china
poland
guatemala
morocco
panama
honduras
czech-republic
israel
portugal
dominican-republic
algeria
switzerland
ireland
kazakhstan
japan
moldova
hungary
afghanistan
ethiopia
paraguay
costa-rica
armenia
oman
bulgaria
myanmar
kyrgyzstan
nigeria
bosnia-and-herzegovina
austria
belarus
el-salvador
north-macedonia
kenya
australia
tunisia
nepal
sudan
libya
puerto-rico
serbia
venezuela
kuwait
denmark
azerbaijan
kosovo
jordan
yemen
lebanon
greece
uzbekistan
albania
united-arab-emirates
croatia
donetsk-pr
south-korea
palestine
cameroon
finland
zambia
senegal
bahrain
ghana
dr-congo
norway
montenegro
syria
angola
slovenia
madagascar
zimbabwe
georgia
malaysia
haiti
qatar
jamaica
malawi
slovakia
mauritania
nicaragua
luxembourg
lithuania
bahamas
mali
namibia
cuba
ivory-coast
guyana
the-gambia
eswatini
suriname
trinidad-and-tobago
hong-kong
somalia
uganda
chad""".splitlines()
countries

['us',
 'brazil',
 'india',
 'mexico',
 'uk',
 'italy',
 'france',
 'spain',
 'peru',
 'iran',
 'colombia',
 'argentina',
 'russia',
 'south-africa',
 'chile',
 'indonesia',
 'ecuador',
 'belgium',
 'iraq',
 'germany',
 'canada',
 'turkey',
 'bolivia',
 'netherlands',
 'philippines',
 'pakistan',
 'ukraine',
 'romania',
 'egypt',
 'sweden',
 'bangladesh',
 'saudi-arabia',
 'china',
 'poland',
 'guatemala',
 'morocco',
 'panama',
 'honduras',
 'czech-republic',
 'israel',
 'portugal',
 'dominican-republic',
 'algeria',
 'switzerland',
 'ireland',
 'kazakhstan',
 'japan',
 'moldova',
 'hungary',
 'afghanistan',
 'ethiopia',
 'paraguay',
 'costa-rica',
 'armenia',
 'oman',
 'bulgaria',
 'myanmar',
 'kyrgyzstan',
 'nigeria',
 'bosnia-and-herzegovina',
 'austria',
 'belarus',
 'el-salvador',
 'north-macedonia',
 'kenya',
 'australia',
 'tunisia',
 'nepal',
 'sudan',
 'libya',
 'puerto-rico',
 'serbia',
 'venezuela',
 'kuwait',
 'denmark',
 'azerbaijan',
 'kosovo',
 'jordan',
 'yemen',
 'leb

In [15]:
base_url = "https://www.worldometers.info/coronavirus/country/"
dfs = []
for country in countries:
    temp = base_url + country
    dfs.append(parseVals(temp, country, 'graph-cases-daily'))

In [16]:
dfs

[       date     value country
 0    Feb 15      null      us
 1    Feb 16         0      us
 2    Feb 17         0      us
 3    Feb 18         0      us
 4    Feb 19         0      us
 ..      ...       ...     ...
 262  Nov 03     94669      us
 263  Nov 04    108554      us
 264  Nov 05    123718      us
 265  Nov 06    132541      us
 266  Nov 07  124390]}      us
 
 [267 rows x 3 columns],
        date    value country
 0    Feb 15     null  brazil
 1    Feb 16     null  brazil
 2    Feb 17     null  brazil
 3    Feb 18     null  brazil
 4    Feb 19     null  brazil
 ..      ...      ...     ...
 262  Nov 03    12920  brazil
 263  Nov 04    23815  brazil
 264  Nov 05    23317  brazil
 265  Nov 06    18247  brazil
 266  Nov 07  21056]}  brazil
 
 [267 rows x 3 columns],
        date    value country
 0    Feb 15     null   india
 1    Feb 16        0   india
 2    Feb 17        0   india
 3    Feb 18        0   india
 4    Feb 19        0   india
 ..      ...      ...     ...
 262

In [17]:
tempdf = pd.concat(dfs)
tempdf

,date,value,country
0,Feb 15,null,us
1,Feb 16,0,us
2,Feb 17,0,us
3,Feb 18,0,us
4,Feb 19,0,us
...,...,...,...
262,Nov 03,14,chad
263,Nov 04,4,chad
264,Nov 05,12,chad
265,Nov 06,9,chad


# Reformat df into pivot table

In [18]:
pivottable = pd.pivot_table(tempdf, values='value', index=['date'], columns=['country'], aggfunc=sum).fillna(0)
pivottable

country,afghanistan,albania,algeria,angola,argentina,armenia,australia,austria,azerbaijan,bahamas,...,uganda,uk,ukraine,united-arab-emirates,us,uzbekistan,venezuela,yemen,zambia,zimbabwe
date,,,,,,,,,,,,,,,,,,,,,
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Apr 01,63,16,131,1,79,39,285,531,61,7,...,0,3917,149,150,27372,9,9,null,0,0
Apr 02,36,18,139,0,132,92,266,418,41,3,...,1,3845,103,210,30702,24,2,null,3,1
Apr 03,8,27,185,0,88,73,140,395,43,0,...,3,4032,175,240,33080,22,7,null,0,0
Apr 04,28,29,80,2,98,34,96,257,78,4,...,0,3384,153,241,34562,39,2,null,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Sep 26,6,108,160,82,11249,429,16,714,108,91,...,146,7579,3833,1078,43582,427,667,1,97,16
Sep 27,35,106,153,46,8841,328,24,662,128,48,...,166,7713,3130,851,34116,501,751,0,29,9
Sep 28,6,132,146,79,11807,174,4,556,38,0,...,247,4662,2671,626,37394,456,837,1,19,4


In [19]:
pivottable.to_csv("corona_country_daily_v3.csv")